## Load Data-**set**

In [2]:
#important libraries
import pandas as pd
import numpy as np
import shutil
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import torch.nn as nn
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize


In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.2 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#define the source  path
source_file_path = 'data.csv'
#define destination file path
destination_file_path = '/content/drive/My Drive/data.csv'
#use shutil to copy the file to the new destination
shutil.copy(source_file_path, destination_file_path)

'/content/drive/My Drive/data.csv'

In [5]:
#define the file path in  Google Drive
file_path = '/content/data.csv'

# read data set
data = pd.read_csv(file_path)

In [6]:
data

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,"['explan', 'edit', 'made', 'usernam', 'hardcor...",0,0,0,0,0,0
1,000103f0d9cfb60f,"['daww', 'match', 'background', 'colour', 'im'...",0,0,0,0,0,0
2,000113f07ec002fd,"['hey', 'man', 'im', 'realli', 'tri', 'edit', ...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"['cant', 'make', 'real', 'suggest', 'improv', ...",0,0,0,0,0,0
4,0001d958c54c6e35,"['sir', 'hero', 'chanc', 'rememb', 'page', 'th...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,"['second', 'time', 'ask', 'view', 'complet', '...",0,0,0,0,0,0
159567,ffea4adeee384e90,"['asham', 'horribl', 'thing', 'put', 'talk', '...",0,0,0,0,0,0
159568,ffee36eab5c267c9,"['spitzer', 'umm', 'there', 'actual', 'articl'...",0,0,0,0,0,0
159569,fff125370e4aaaf3,"['look', 'like', 'actual', 'put', 'speedi', 'f...",0,0,0,0,0,0


## Bag of words

In [7]:
#create a CountVectorizer  used to convert the text data into a numerical format
vectorizer = CountVectorizer()
#fit the vectorizer mean analyze the text data and learn the unique word vocabulary
#transformation mean count the frequency and create a numerical matrix
x = vectorizer.fit_transform(data['comment_text'])
#represent the bag of word matrix in array by using  toarray
#it need more RAM i have
#bow_array = x.toarray()


In [8]:
# calculate sum the frequency of numerical matrix
frequency= np.sum(x, axis=0)
#covert matrix to array
array= np.squeeze(np.asarray(frequency))
#get the words name in the bag of word using get_feature_names_out().
name= vectorizer.get_feature_names_out()
#make list of tuple have words and this number
words= list(zip(name, array))
#sort frequency in descending order
sort= sorted(words, key=lambda item: item[1], reverse=True)
#get top 25 words
sort[:25]

[('articl', 72948),
 ('page', 56443),
 ('edit', 40526),
 ('wikipedia', 38380),
 ('use', 34798),
 ('talk', 33885),
 ('like', 30168),
 ('one', 29932),
 ('pleas', 29781),
 ('would', 29212),
 ('delet', 26199),
 ('dont', 26106),
 ('thank', 23738),
 ('sourc', 23402),
 ('see', 22612),
 ('think', 21870),
 ('make', 20777),
 ('also', 20550),
 ('know', 20511),
 ('im', 19471),
 ('time', 18879),
 ('peopl', 18738),
 ('say', 17920),
 ('get', 17913),
 ('go', 17540)]

            After make bag of word and covert text data to frequency numerical form extracted the top  words are articale , page and edit

## Word Embedding

In [9]:
data['comment_tokens'] = data['comment_text']
#train the  model on  text data
model = Word2Vec(sentences=data['comment_tokens'], vector_size=100, window=5, min_count=1, sg=0)
#function to calculate comment vectors using CBOW
#this funcation take two input the data and model
def comment_vector(comment, model):
    #create  empty list  vectors and check if words in the comment  in the model or not
    vectors = [model.wv[word] for word in comment if word in model.wv.index_to_key]
    #if words are in the model calculate the comment vector
    #average is the sum of all the word vectors divided by the number of word vectors
    if vectors:
        return sum(vectors) / len(vectors)
    #if not return a vector of zeros
    else:
        return [0.0] * model.vector_size

#calculate comment vectors for data by function
data['comment_vectors'] = data['comment_tokens'].apply(lambda x: comment_vector(x, model))

In [10]:
data

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment_tokens,comment_vectors
0,0000997932d777bf,"['explan', 'edit', 'made', 'usernam', 'hardcor...",0,0,0,0,0,0,"['explan', 'edit', 'made', 'usernam', 'hardcor...","[-0.29150224, -0.45998967, 0.530248, 0.4055003..."
1,000103f0d9cfb60f,"['daww', 'match', 'background', 'colour', 'im'...",0,0,0,0,0,0,"['daww', 'match', 'background', 'colour', 'im'...","[-0.22925748, -0.51653653, 0.36996064, 0.30296..."
2,000113f07ec002fd,"['hey', 'man', 'im', 'realli', 'tri', 'edit', ...",0,0,0,0,0,0,"['hey', 'man', 'im', 'realli', 'tri', 'edit', ...","[-0.28178683, -0.4601001, 0.5901394, 0.4115298..."
3,0001b41b1c6bb37e,"['cant', 'make', 'real', 'suggest', 'improv', ...",0,0,0,0,0,0,"['cant', 'make', 'real', 'suggest', 'improv', ...","[-0.2740721, -0.44926575, 0.5700505, 0.4135008..."
4,0001d958c54c6e35,"['sir', 'hero', 'chanc', 'rememb', 'page', 'th...",0,0,0,0,0,0,"['sir', 'hero', 'chanc', 'rememb', 'page', 'th...","[-0.2595137, -0.4817929, 0.46911374, 0.3245766..."
...,...,...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,"['second', 'time', 'ask', 'view', 'complet', '...",0,0,0,0,0,0,"['second', 'time', 'ask', 'view', 'complet', '...","[-0.27876362, -0.46127194, 0.5840415, 0.425025..."
159567,ffea4adeee384e90,"['asham', 'horribl', 'thing', 'put', 'talk', '...",0,0,0,0,0,0,"['asham', 'horribl', 'thing', 'put', 'talk', '...","[-0.29319838, -0.49675012, 0.20536669, 0.36758..."
159568,ffee36eab5c267c9,"['spitzer', 'umm', 'there', 'actual', 'articl'...",0,0,0,0,0,0,"['spitzer', 'umm', 'there', 'actual', 'articl'...","[-0.31234968, -0.4207352, 0.53905445, 0.385290..."
159569,fff125370e4aaaf3,"['look', 'like', 'actual', 'put', 'speedi', 'f...",0,0,0,0,0,0,"['look', 'like', 'actual', 'put', 'speedi', 'f...","[-0.23522177, -0.4181437, 0.596086, 0.3365914,..."


            i added new column comment_vectors it contain average of each words in comment  in CBOW model

## Bidirectional Encoder Representations from Transformers

In [11]:
#prepares text data and label for classification task using bert model
#comments is the train coulmn
comments = data['comment_text'].astype(str)
#labels are targets coulmns
labels = data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values
#use train_test_split function from Scikit-Learn to split the data into train and test sets.
#80% for train and 20% for test and random state 42
train_comments, test_comments, train_labels, test_labels = train_test_split(
    comments, labels, test_size=0.2, random_state=42
)
#tokenize(break input text into smaller units called tokens) the text data in train comment and test
#truncation ensures that tokens longer than the maximum input length are truncated
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
train_encodings = tokenizer(train_comments.tolist(), truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(test_comments.tolist(), truncation=True, padding=True, return_tensors='pt')

#convert label to PyTorch tensor
train_labels = torch.tensor(train_labels, dtype=torch.float32)
test_labels = torch.tensor(test_labels, dtype=torch.float32)

#create DataLoader( allow to iterate through your train and test data in batches) for train and test
#input_ids and  attention_mask used to create data loader during train and test
#datasets can be used to create data loaders for batching during training and testing.
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)
#number of data samples processed in each iteration during training.
batch_size = 16


In [12]:
#checks if a CUDA-enabled GPU is available (parallel computing platform and application programming ).
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [13]:
#creat pretrained BERT model for sequence classification with 6 labels
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)
#move model to device
model.to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [14]:
# training parameters
epochs = 4
learning_rate = 2e-5

#Create DataLoader for training and testing
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#create optimizer used for training deep learning models by using learning rate and epoch
optimizer = AdamW(model.parameters(), lr=learning_rate)
# loss function training a neural network model.
criterion = nn.BCEWithLogitsLoss()



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
#training data using the train_loader steps
#1-calculate gradients,
#2- update the model's weights( update loss funcation).
#3- scheduler is used to  the learning rate as .
#4- print the average training loss for each epoch and save the trained model.
# training loop for a neural network model
for epoch in range(epochs):
  #model in training mode
    model.train()
    total_loss = 0
    accumulated_steps = 0
   #get input_ids, attention_mask and labels from the batch
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        #zero out gradients from the previous batch
        optimizer.zero_grad()
        #forward pass through the model to get predictions
        outputs = model(input_ids, attention_mask=attention_mask)
        #calculate the loss between model predictions and true labels
        loss = criterion(outputs.logits, labels)
        #accumulate the batch loss to calculate the average later
        total_loss += loss.item()
        #backpropagate gradients through the network
        loss.backward()
        #increment the accumulated steps counter
        accumulated_steps += 1
        #check if it's time to update the model weights
        if accumulated_steps == accumulation_steps:
            #update model weights every accumulation_steps
            optimizer.step()
            #reset the accumulated steps counter
            accumulated_steps = 0  # Reset accumulated steps

    #if there are remaining accumulated gradients, update the weights
    if accumulated_steps > 0:
        optimizer.step()
    #calculate the average training loss for the epoch
    avg_train_loss = total_loss / len(train_loader)
    #rint the average training loss for the current epoch
    print(f"Epoch {epoch + 1}/{epochs}, Average Training Loss: {avg_train_loss:.4f}")

# Save the trained model
model.save_pretrained("bert_toxicity_model")


In [ ]:
#evualte test data steps
#1- calculate prediction using the model
#2- compare this prediction to true labels
#3- apply a threshold of 0.5 for each label
#4- generate a classification report to print   precision, recall, and F1-score

# model.eval() to evaluation mode
model.eval()
#empty list to store predictions and true label
all_preds = []
all_labels = []

#disable gradient computation during evaluation
with torch.no_grad():
    #iterate over batches in the test data loader
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
       #forward pass through the model to get predictions (logits)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        #apply the sigmoid activation function to obtain probability scores
        sigmoid = nn.Sigmoid()
        preds = sigmoid(logits)
        #extend the lists with predictions and true labels
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

#make list to numpy array to get accuracy of the model
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)
#get numers from the array  which > threshold
threshold = 0.5
predicted_labels = (all_preds > threshold).astype(int)

#print summary of the model performance for each label
report = classification_report(all_labels, predicted_labels, target_names=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])
print(report)


In [ ]:
#the code not complete run at training model (crashed) because it need RAM over 12.5